### LightFM recommendation for homepage


In [25]:
import os
import random
from scipy import sparse
import pandas as pd
import numpy as np

import lightfm
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score

from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
books=pd.read_csv("Booksbx.csv",sep=";",encoding="latin-1",encoding_errors= 'replace',error_bad_lines=False)#,nrows=20000)
ratings=pd.read_csv("BookRatingsbx.csv",sep=";",encoding="utf-8",encoding_errors= 'replace',error_bad_lines=False)#,nrows=20000)


b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/var/folders/7h/btw3443n6hx6nbd24dh7dpjc0000gn/T/ipykernel_21175/819840318

In [5]:
df_playlist = ratings

In [6]:
df_playlist =df_playlist.groupby('ISBN').filter(lambda x: len(x)>=100)
df_playlist

,User-ID,ISBN,Book-Rating
2,276727,0446520802,0
8,276744,038550120X,7
10,276746,0425115801,0
11,276746,0449006522,0
12,276746,0553561618,0
...,...,...,...
1149749,276690,0064400557,0
1149761,276704,0345386108,6
1149768,276704,0446605409,0
1149771,276704,0743211383,7


In [7]:
df_playlist = df_playlist[df_playlist.groupby('User-ID').ISBN.transform('nunique')>=10]
df_playlist

,User-ID,ISBN,Book-Rating
796,277042,0061097101,0
799,277042,0312983271,0
800,277042,0380731851,0
801,277042,0446605484,7
802,277042,0446611212,8
...,...,...,...
1149553,276680,0446670251,0
1149564,276680,0452283205,7
1149577,276680,0679731725,0
1149581,276680,0679781587,9


In [8]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [9]:
def create_user_dict(interactions):
    
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [10]:
def create_item_dict(df,id_col,name_col):
    
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [11]:
def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4):
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

In [12]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
  
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
                       .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Ratings:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [13]:
interactions = create_interaction_matrix(df = df_playlist, user_col = "User-ID", item_col = 'ISBN', 
                                         rating_col = 'Book-Rating', norm= False, threshold = None)
interactions.head()

ISBN,002542730X,0060008032,0060096195,006016848X,0060173289,0060175400,006019491X,0060199652,0060391626,0060392452,...,1558744630,1558745157,1559029838,1573225517,1573225789,1573227331,1573229326,1573229571,1592400876,1878424319
User-ID,,,,,,,,,,,,,,,,,,,,,
243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
user_dict = create_user_dict(interactions=interactions)
print(user_dict)

{243: 0, 254: 1, 507: 2, 638: 3, 882: 4, 1131: 5, 1424: 6, 1435: 7, 1733: 8, 1848: 9, 1903: 10, 2033: 11, 2276: 12, 2358: 13, 2766: 14, 2793: 15, 2891: 16, 2977: 17, 3145: 18, 3167: 19, 3363: 20, 3371: 21, 3373: 22, 3556: 23, 4017: 24, 4092: 25, 4622: 26, 4795: 27, 4938: 28, 5037: 29, 5439: 30, 5483: 31, 5539: 32, 5582: 33, 5609: 34, 5903: 35, 6242: 36, 6251: 37, 6323: 38, 6431: 39, 6543: 40, 6563: 41, 6575: 42, 6577: 43, 6772: 44, 6900: 45, 7082: 46, 7125: 47, 7158: 48, 7283: 49, 7286: 50, 7346: 51, 7569: 52, 7620: 53, 7887: 54, 7915: 55, 7958: 56, 8019: 57, 8066: 58, 8067: 59, 8187: 60, 8234: 61, 8245: 62, 8253: 63, 8284: 64, 8362: 65, 8454: 66, 8487: 67, 8681: 68, 8782: 69, 8801: 70, 8870: 71, 8936: 72, 8937: 73, 9141: 74, 9417: 75, 9747: 76, 9856: 77, 9908: 78, 10030: 79, 10314: 80, 10354: 81, 10447: 82, 10560: 83, 10819: 84, 11224: 85, 11245: 86, 11400: 87, 11601: 88, 11676: 89, 11724: 90, 11993: 91, 12538: 92, 12569: 93, 12824: 94, 12982: 95, 13082: 96, 13273: 97, 13552: 98, 1358

In [15]:
df_playlist


,User-ID,ISBN,Book-Rating
796,277042,0061097101,0
799,277042,0312983271,0
800,277042,0380731851,0
801,277042,0446605484,7
802,277042,0446611212,8
...,...,...,...
1149553,276680,0446670251,0
1149564,276680,0452283205,7
1149577,276680,0679731725,0
1149581,276680,0679781587,9


In [16]:
df_item = books

In [17]:
item_dict = create_item_dict(df=df_item, id_col='ISBN', name_col='Book-Title')
print(item_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
x = sparse.csr_matrix(interactions.values)
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.2, random_state=None)

In [19]:
%time
model = runMF(interactions = train,
                 n_components = 30,
                 loss = 'warp',
                 k = 15,
                 epoch = 30,
                 n_jobs = 4)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [20]:
train_auc = auc_score(model, train, num_threads=4).mean()
print('Train AUC: %s' % train_auc)

Train AUC: 0.97884953


In [21]:
test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
print('Test AUC: %s' % test_auc)

Test AUC: 0.97691435


In [22]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

In [23]:
print('train Precision %.2f, test Precision %.2f.' % (train_precision, test_precision))

train Precision 0.37, test Precision 0.16.


In [26]:
rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = 55492, 
                                      user_dict = user_dict,
                                      item_dict = item_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Ratings:
1- Call of the Wild
2- The Lost Boy: A Foster Child's Search for the Love of a Family
3- Nicolae: The Rise of Antichrist (Left Behind No. 3)
4- Schindler's List
5- The Perks of Being a Wallflower
6- The Shelters of Stone (Earth's Children Series, No 5)
7- Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
8- Anne of Green Gables (Anne of Green Gables Novels (Paperback))
9- Girl with a Pearl Earring
10- Dude, Where's My Country?
11- The Bridges of Madison County
12- Harry Potter and the Order of the Phoenix (Book 5)
13- Harry Potter and the Goblet of Fire (Book 4)
14- Harry Potter and the Prisoner of Azkaban (Book 3)
15- Harry Potter and the Chamber of Secrets (Book 2)
16- Portrait of a Killer: Jack the Ripper-- Case Closed (Berkley True Crime)
17- Sisterhood of the Traveling Pants
18- Bleachers
19- The King of Torts
20- The Thorn Birds
21- The Big Bad Wolf: A Novel
22- The Hours: A Novel
23- Mrs Dalloway
24- On the Road
25- The Bad Beginning (A Series of Un

In [27]:
rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = 62891, 
                                      user_dict = user_dict,
                                      item_dict = item_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Ratings:
1- The Lost Boy: A Foster Child's Search for the Love of a Family
2- ANGELA'S ASHES
3- The English Patient
4- American Psycho (Vintage Contemporaries)
5- The Bridges of Madison County
6- Hannibal
7- Red Dragon
8- The Hot Zone
9- Rising Sun
10- Congo
11- The Andromeda Strain
12- Jurassic Park
13- Sphere
14- The Lovely Bones: A Novel
15- Bridget Jones's Diary
16- Of Mice and Men (Penguin Great Books of the 20th Century)
17- Love in the Time of Cholera (Penguin Great Books of the 20th Century)
18- The Perfect Storm : A True Story of Men Against the Sea
19- Pigs in Heaven

 Recommended Items:
1- The Firm
2- Silence of the Lambs
3- The Green Mile
4- Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson
5- The Hunt for Red October
6- Don't Sweat the Small Stuff and It's All Small Stuff : Simple Ways to Keep the Little Things from Taking Over Your Life (Don't Sweat the Small Stuff Series)
7- SHIPPING NEWS
8- On the Road
9- Circle of Friends
10- The Client
